In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-11-18 10:34:16--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2025-11-18 10:34:16 (116 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r' , encoding='utf-8') as file:
  text = file.read()

In [ ]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
len(text)

1115394

In [ ]:
set(text)

{'\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
chars = sorted(list(set(text)))

In [ ]:
len(chars)

65

In [ ]:
# Vocabulary size = 65
# BATCH
# Sequence/block_size
# P(next_token | current_token)

In [ ]:
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [ ]:
stoi = {ch : i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

In [ ]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [ ]:
print("hello")

print(encode("abcdefgh"))

hello
[39, 40, 41, 42, 43, 44, 45, 46]


In [ ]:
print(decode(encode("abcdefgh")))

abcdefgh


In [ ]:
import torch
data = torch.tensor(encode(text), dtype= torch.long)
print(data.shape)

torch.Size([1115394])


In [ ]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [ ]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# Vocabulary size = 65
# BATCH
# Sequence/block_size = 8
# target_size = 1
# P(next_token | current_token)

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
batch_size = 4
block_size = 8

def get_batches(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y


In [ ]:
xb, yb = get_batches('train')

In [ ]:
print(xb.shape, yb.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [ ]:
xb

tensor([[57,  1, 43, 52, 43, 51, 47, 43],
        [ 1, 40, 43,  1, 53, 40, 43, 42],
        [47, 57, 46,  5, 42,  6,  0, 24],
        [47, 52,  1, 39,  1, 51, 47, 50]])

In [ ]:
yb

tensor([[ 1, 43, 52, 43, 51, 47, 43, 57],
        [40, 43,  1, 53, 40, 43, 42, 47],
        [57, 46,  5, 42,  6,  0, 24, 43],
        [52,  1, 39,  1, 51, 47, 50, 43]])

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


In [ ]:
class BiGramLangauageModel(nn.Module):
  # constructor
  # forward pass (predict)
  # generate (auto-regressive in nature )

  def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits/probabilities for the next token from a lookup table
        # Each character (token) is mapped to a vocab_size-dimensional vector.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # It initializes an embedding layer.

  def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        # one char needs 65 probabs.
        # B x T chars needs 65 probabs each
        # thus total size = B x T x C

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

# auto-regressive in nature
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)

      logits = logits[:, -1, :] # becomes (B, C)
      # logits have shape B, vocab_size or B x 65
      #apply softmax function
      probabs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probabs, num_samples=1)
      idx = torch.cat([idx, idx_next], dim=1)
    return idx


In [ ]:
m = BiGramLangauageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8649, grad_fn=<NllLossBackward0>)

JeeWvpIS,XMEDtHTWseVts!xeDZRjTPChtPt!m!!x;hrcchNZNmW3uOfUUKtHfYE;Wv!xL.:!xF;BRXS$JBbp$ck$NCZwTETgTZD


In [ ]:
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8649, grad_fn=<NllLossBackward0>)


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batches('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.535292148590088


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



x'sXUtmOUjUkjEG'FQ
JCgwQKsHB;HHa!'N?UxXipNRf$aiHc&T3iccfceuuk3ew3j-:DWOWpydcdUFBfuOAevL.z''?fnsHUmkf hwAlpMw'x'A$zr;
xuf;vLKfQf!CRR:Q&xQwQAGstSEF;eVApbSTLqgXW3NCemPwprMUpO maj'a'mHZ,cg!jRjYgn
f lDAGO-C ;f'tukvqI-bgiJPnkjlWVprNMRTjlOWuboDHMqXUYWKT!VY?kzEvjl
umDfwry
jlqXZjlaZZzU,pyxeaRS-ag$NClC:hYrtmryhstbp$ XWK;ZsAy3aclWgv$-VA;BdqRiAjltoS$qx'WUtmZ?U?rNhwjhwJnmt;xW;TMw$'z?D.!ymxc,:IRoxn-.
!zG&DSpbo bpO3TTe$IwBNC TnkjExzG.IxL$3&ERcfg&ctCTva-S,H-;Rqa-,edvL&zqXOVLVY
wBxewyXUhv,&aRzk3zM.3dqM3Rlg&.r!m


In [ ]:
torch.zeros((1, 1), dtype=torch.long)

tensor([[0]])